In [1]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from holidays.countries import US
import numpy as np

### DEFINE

In [2]:
# 讀取檔案位置

# 初始年月份
set_data_year = 2023 #start_year
set_data_month = 1 #start_month
end_year = 2023
end_month = 6

# 檔案位置
Site = '../RawData/'

# 檔案輸出位置
custom_output_path_dfTime = './dfTime/test'
custom_output_path_dfTLC = './dfTLC/test'

# 年月
start_year = set_data_year
start_month = set_data_month

datasite_fhvhv = []
datasite_yellow = []
for year in range(start_year, end_year+1):  # 修改为你的年份范围
    if year == start_year:
        if year < end_year :
            for month in range(start_month, 13):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)                
        elif year == end_year:
            for month in range(start_month, end_month+1):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)    
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   
    elif year > start_year:
        if year < end_year :
            for month in range(1, 13):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   
        elif year == end_year:
            for month in range(1, end_month+1):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)         
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   



print("done時間:", datetime.datetime.now())

done時間: 2023-09-13 10:05:00.000113


### 留下需要特徵 & 移除不在當月資料
    HVFHV: 'hvfhs_license_num','pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','trip_miles','trip_time','base_passenger_fare'
    
    Yellow: 'tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID','fare_amount'

In [3]:
# 記錄開始時間
start_time = time.time()


# 建立一個空的DataFrame來存放合併後的數據
df_fhvhv = pd.DataFrame()
df_yellow = pd.DataFrame()

data_year = set_data_year
data_month = set_data_month
temp0_total_fhvfv = 0
temp1_total_fhvfv = 0
temp0_total_yellow = 0
temp1_total_yellow = 0
# 逐一讀取並合併所有Parquet檔案
print('fhvfv==初始筆數=剩餘筆數=移除佔比')
for file in datasite_fhvhv:
    # 使用pandas的read_parquet函數讀取Parquet檔案並將其轉換為DataFrame
    df = pd.read_parquet(file)
    temp_0 = df.shape[0]
    temp0_total_fhvfv += temp_0
    # 留下需要的欄位
    df =  df[['hvfhs_license_num','pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','trip_miles','trip_time','shared_match_flag']]
    # 移除非當月的data
    df =  df[(df['pickup_datetime'].dt.year == data_year) & (df['pickup_datetime'].dt.month == data_month)]
    temp_1 = df.shape[0]
    temp1_total_fhvfv += temp_1
    # 查看剩餘資料量
    if temp_0 > temp_1 :
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,round(100-(temp_1/temp_0*100),5),"%")
    else:
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,'=0%')
    # 年月處理
    if data_month == 12:
        data_year += 1
        data_month = 1
    else :
        data_month += 1
    # 將數據合併到已有的DataFrame中
    df_fhvhv = pd.concat([df_fhvhv, df], ignore_index=True)
    
    

data_year = set_data_year
data_month = set_data_month
print('yellow==初始筆數=剩餘筆數=移除佔比')
for file in datasite_yellow:
    # 使用pandas的read_parquet函數讀取Parquet檔案並將其轉換為DataFrame
    df = pd.read_parquet(file)
    temp_0 = df.shape[0]
    temp0_total_yellow += temp_0
    # 留下需要的欄位
    df =  df[['tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID']]
    # 移除非當月的data
    df =  df[(df['tpep_pickup_datetime'].dt.year == data_year) & (df['tpep_pickup_datetime'].dt.month == data_month)]
    temp_1 = df.shape[0]
    temp1_total_yellow += temp_1
    # 查看剩餘資料量
    if temp_0 > temp_1 :
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,'=',round(100-(temp_1/temp_0*100),5),"%")
    else:
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,'=0%')        
    # 年月處理
    if data_month == 12:
        data_year += 1
        data_month = 1
    else :
        data_month += 1
    # 將數據合併到已有的DataFrame中
    df_yellow = pd.concat([df_yellow, df], ignore_index=True)
    
    
print(f'fhvfv={start_year}=初始總筆數=剩餘總筆數=移除佔比')
if temp0_total_fhvfv > temp1_total_fhvfv :
    print('\==',temp0_total_fhvfv,'=',temp1_total_fhvfv,'=',round(100-(temp1_total_fhvfv/temp0_total_fhvfv*100),5),"%")
else:
    print('\==',temp0_total_fhvfv,'=',temp1_total_fhvfv,'=0%')

print(f'yellow={start_year}=初始總筆數=剩餘總筆數=移除佔比')
if temp0_total_yellow > temp1_total_yellow :
    print('\==',temp0_total_yellow,'=',temp1_total_yellow,'=',round(100-(temp1_total_yellow/temp0_total_yellow*100),5),"%")
else:
    print('\==',temp0_total_yellow,'=',temp1_total_yellow,'=0%')

# 記錄結束時間
end_time = time.time()
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")


#清理暫存
del data_month,data_year,df,temp_0,temp_1,temp0_total_yellow,temp1_total_yellow
print("done時間:", datetime.datetime.now())

fhvfv==初始筆數=剩餘筆數=移除佔比
2023 = 1 = 18479031 = 18479031 =0%
2023 = 2 = 17960971 = 17960971 =0%
2023 = 3 = 20413539 = 20413539 =0%
2023 = 4 = 19144903 = 19144903 =0%
2023 = 5 = 19847676 = 19847676 =0%
2023 = 6 = 19366619 = 19366619 =0%
yellow==初始筆數=剩餘筆數=移除佔比
2023 = 1 = 3066766 = 3066718 = 0.00157 %
2023 = 2 = 2913955 = 2913900 = 0.00189 %
2023 = 3 = 3403766 = 3403577 = 0.00555 %
2023 = 4 = 3288250 = 3288155 = 0.00289 %
2023 = 5 = 3513649 = 3513599 = 0.00142 %
2023 = 6 = 3307234 = 3307193 = 0.00124 %
fhvfv=2023=初始總筆數=剩餘總筆數=移除佔比
\== 115212739 = 115212739 =0%
yellow=2023=初始總筆數=剩餘總筆數=移除佔比
\== 19493620 = 19493142 = 0.00245 %

程式執行花費的時間： 119.99 秒
done時間: 2023-09-13 10:07:02.026695


In [4]:
df_fhvhv

,hvfhs_license_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,shared_match_flag
0,HV0003,2023-01-01 00:19:38,2023-01-01 00:48:07,48,68,0.940,1709,N
1,HV0003,2023-01-01 00:58:39,2023-01-01 01:33:08,246,163,2.780,2069,N
2,HV0003,2023-01-01 00:20:27,2023-01-01 00:37:54,9,129,8.810,1047,N
3,HV0003,2023-01-01 00:41:05,2023-01-01 00:48:16,129,129,0.670,431,N
4,HV0003,2023-01-01 00:52:47,2023-01-01 01:04:51,129,92,4.380,724,N
...,...,...,...,...,...,...,...,...
115212734,HV0005,2023-06-30 23:15:16,2023-06-30 23:19:30,38,191,1.027,254,N
115212735,HV0005,2023-06-30 23:24:46,2023-06-30 23:47:49,191,216,8.842,1383,N
115212736,HV0005,2023-06-30 23:58:44,2023-07-01 00:05:15,215,10,0.360,391,N
115212737,HV0003,2023-06-30 23:32:06,2023-06-30 23:41:26,42,116,1.570,560,N


In [5]:
df_yellow

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID
0,2023-01-01 00:32:10,2023-01-01 00:40:36,0.97,161,141
1,2023-01-01 00:55:08,2023-01-01 01:01:27,1.10,43,237
2,2023-01-01 00:25:04,2023-01-01 00:37:49,2.51,48,238
3,2023-01-01 00:03:48,2023-01-01 00:13:25,1.90,138,7
4,2023-01-01 00:10:29,2023-01-01 00:21:19,1.43,107,79
...,...,...,...,...,...
19493137,2023-06-30 23:30:21,2023-07-01 00:00:46,9.20,42,148
19493138,2023-06-30 23:34:22,2023-07-01 00:32:59,20.20,132,74
19493139,2023-06-30 23:45:00,2023-07-01 00:10:00,6.16,256,140
19493140,2023-06-30 23:13:38,2023-06-30 23:51:19,11.65,91,246


### 特徵整理,統一命名
HVFHV:'hvfhs_license_num','pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','trip_miles','trip_time'
Yellow:'tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID'


    newname (HVFHV/Yellow/Green)
    1.TLC 許可證號 : Name(hvfhs_license_num/--/--)
        *改為名稱標記 HV0003-uber/HV0005-lyft/yellow
    2.上車時間 : BoardingTime (pickup_datetime/tpep_pickup_datetime/lpep_pickup_datetime)
    3.下車時間 : DisembarkingTime (dropoff_datetime/tpep_dropoff_datetime/lpep_dropoff_datetime)
    4.上車地點ID : PULocationID (PULocationID/PULocationID/PULocationID)
    5.下車地點ID : DOLocationID (DOLocationID/DOLocationID/DOLocationID)
    6.行駛里程 : TripMiles (trip_miles/trip_distance/trip_distance)
    7.行駛時間(秒) : TripTime (trip_time/--/--)
        *驗證是否正確?
        *下車時間-上車時間
    8.共乘 : shared_match_flag (shared_match_flag/--/--)
    9.節日 : is_holiday(--/--/--)
    10.星期 : weekday(--/--/--)

In [6]:
# 記錄開始時間
start_time = time.time()

df_TLC = pd.DataFrame()
#名稱統一
rename_dict_fhvhv = {'hvfhs_license_num':'Name','pickup_datetime': 'BoardingTime', 'dropoff_datetime': 'DisembarkingTime', 'PULocationID': 'PULocationID','DOLocationID':'DOLocationID', 'trip_miles': 'TripMiles', 'trip_time': 'TripTime', 'base_passenger_fare': 'BaseFare','shared_match_flag':'shared_match_flag'}
df_fhvhv.rename(columns=rename_dict_fhvhv, inplace=True)
rename_dict_yellow = {'tpep_pickup_datetime': 'BoardingTime', 'tpep_dropoff_datetime': 'DisembarkingTime', 'PULocationID': 'PULocationID','DOLocationID':'DOLocationID', 'trip_distance': 'TripMiles', 'fare_amount': 'BaseFare'}
df_yellow.rename(columns=rename_dict_yellow, inplace=True)

#fhvhv_Name
df_fhvhv.loc[df_fhvhv['Name'] == 'HV0003', 'Name'] = 'uber'
df_fhvhv.loc[df_fhvhv['Name'] == 'HV0005', 'Name'] = 'lyft'

#yellow 新增特徵 TripTime,Name
df_yellow['TripTime'] = (df_yellow['DisembarkingTime'] - df_yellow['BoardingTime']).dt.total_seconds()
df_yellow['Name'] = 'yellow'
df_yellow['shared_match_flag'] = 'N'
#yellow合併到fhvhv
df_yellow = df_yellow.reindex(columns = df_fhvhv.columns)
df_TLC = pd.concat([df_fhvhv, df_yellow], ignore_index=True)


# 記錄結束時間
end_time = time.time()
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("程式執行花費的時間：", round(execution_time,2), "秒")

#清理暫存
del df_yellow ,df_fhvhv
print("done時間:", datetime.datetime.now())

程式執行花費的時間： 18.57 秒
done時間: 2023-09-13 10:07:21.599039


### 紀錄初始資料筆數

In [7]:
#紀錄初始資料筆數
init_df_TLC_shape = df_TLC.shape
init_df_uber_shape = df_TLC[df_TLC['Name']=='uber'].shape
init_df_lyft_shape = df_TLC[df_TLC['Name']=='lyft'].shape
init_df_yellow_shape = df_TLC[df_TLC['Name']=='yellow'].shape
print('Name=init_data_row=init_data_column')
print(f'TLC={init_df_TLC_shape[0]}={init_df_TLC_shape[1]}')
print(f'uber={init_df_uber_shape[0]}={init_df_uber_shape[1]}')
print(f'lyft={init_df_lyft_shape[0]}={init_df_lyft_shape[1]}')
print(f'yellow={init_df_yellow_shape[0]}={init_df_yellow_shape[1]}')

print("\ndone時間:", datetime.datetime.now())

Name=init_data_row=init_data_column
TLC=134705881=8
uber=83502693=8
lyft=31710046=8
yellow=19493142=8

done時間: 2023-09-13 10:08:02.682531


### 缺失值確認

In [8]:
print(df_TLC.isnull().sum())

Name                 0
BoardingTime         0
DisembarkingTime     0
PULocationID         0
DOLocationID         0
TripMiles            0
TripTime             0
shared_match_flag    0
dtype: int64


### 移除非目標車商

In [9]:
#查看目前有的車商
print(df_TLC['Name'].unique())

print("\ndone時間:", datetime.datetime.now())

['uber' 'lyft' 'yellow']

done時間: 2023-09-13 10:08:14.087599


### 移除上車地點 PULocationID 未知 (264,265)

In [10]:
#筆數計算
print('264筆數=265筆數=總占比') 
print(df_TLC[(df_TLC['PULocationID'] == 264)].shape[0],'=', df_TLC[(df_TLC['PULocationID'] == 265)].shape[0],'=', 
       round(df_TLC[(df_TLC['PULocationID'] == 264) | (df_TLC['PULocationID'] == 265)].shape[0] / init_df_TLC_shape[0]*100,4),'%'  )

print("\ndone時間:", datetime.datetime.now())

264筆數=265筆數=總占比
208344 = 21243 = 0.1704 %

done時間: 2023-09-13 10:08:14.789280


In [11]:
df_TLC = df_TLC[~df_TLC['PULocationID'].isin([264, 265])]

#check-必須為0
print('上車未知剩餘筆數 = ',df_TLC[(df_TLC['PULocationID'] == 264) | (df_TLC['PULocationID'] == 265)].shape[0])
print("\ndone時間:", datetime.datetime.now())

上車未知剩餘筆數 =  0

done時間: 2023-09-13 10:08:33.743208


### 移除不符合的共乘資料
    UBER 2020/3/17-2022/6/20
    LYFT 2020/3/17-2021/8/18
         2023/5/12-

In [12]:
#uber,lyft
# 定義起始日期和結束日期
uber_start_date = pd.to_datetime('2020-03-17')
uber_end_date = pd.to_datetime('2022-06-20')
lyft_start_date = pd.to_datetime('2020-03-17')
lyft_end_date = pd.to_datetime('2021-08-18')
lyft_start_date1 = pd.to_datetime('2023-05-12')

#筆數計算
temp_uber = ((df_TLC['Name'] == 'uber') & (df_TLC['shared_match_flag'] == 'Y') & (df_TLC['BoardingTime'].between(uber_start_date, uber_end_date))).sum()

temp_lyft = ((df_TLC['Name'] == 'lyft') & (df_TLC['shared_match_flag'] == 'Y') & ((df_TLC['BoardingTime'].between(lyft_start_date, lyft_end_date))|(df_TLC['BoardingTime'] >= lyft_start_date1 ))).sum()
print('/=uber=lyft')
print('data=',temp_uber,'=',temp_lyft)
print('占比=',round(temp_uber / init_df_uber_shape[0]*100,4),'%','=',round(temp_lyft / init_df_lyft_shape[0]*100,4),'%' )

print("\ndone時間:", datetime.datetime.now())

#清理暫存
del temp_uber ,temp_lyft

/=uber=lyft
data= 0 = 3077
占比= 0.0 % = 0.0097 %

done時間: 2023-09-13 10:08:57.124180


In [13]:
# 過濾條件：shared_match_flag為Y且BoardingTime介於指定日期之間
df_TLC = df_TLC[~ ((df_TLC['Name'] == 'uber') & (df_TLC['shared_match_flag'] == 'Y') & (df_TLC['BoardingTime'].between(uber_start_date, uber_end_date)))]
df_TLC = df_TLC[~ ((df_TLC['Name'] == 'lyft') & (df_TLC['shared_match_flag'] == 'Y') & ((df_TLC['BoardingTime'].between(lyft_start_date, lyft_end_date))|(df_TLC['BoardingTime'] >= lyft_start_date1 )))]


#check-必須為0
print('uber不符合的共乘資料剩餘筆數 = ',df_TLC[(df_TLC['Name'] == 'uber') & (df_TLC['shared_match_flag'] == 'Y') & (df_TLC['BoardingTime'].between(uber_start_date, uber_end_date))].shape[0]  )
print('lyft不符合的共乘資料剩餘筆數 = ',df_TLC[(df_TLC['Name'] == 'lyft') & (df_TLC['shared_match_flag'] == 'Y') & ((df_TLC['BoardingTime'].between(lyft_start_date, lyft_end_date))|(df_TLC['BoardingTime'] >= lyft_start_date1 ))].shape[0]  )
print("\ndone時間:", datetime.datetime.now())

#清理暫存
del uber_start_date ,uber_end_date,lyft_start_date,lyft_end_date,lyft_start_date1

uber不符合的共乘資料剩餘筆數 =  0
lyft不符合的共乘資料剩餘筆數 =  0

done時間: 2023-09-13 10:10:38.350650


###　不處理  trip_time != DisembarkingTime - BoardingTime
### 移除 TripTime <= 0

In [14]:
#記錄開始時間
start_time = time.time()

#印出資訊
temp_uber = ((df_TLC['Name'] == 'uber') & (df_TLC['TripTime'] <= 0)).sum()
temp_lyft = ((df_TLC['Name'] == 'lyft') & (df_TLC['TripTime'] <= 0)).sum()
temp_yellow = ((df_TLC['Name'] == 'yellow') & (df_TLC['TripTime'] <= 0)).sum()

print('\=uber=lyft=yellow')
print('data=',temp_uber,'=',temp_lyft,'=',temp_yellow)
print('占比=',round(temp_uber / init_df_uber_shape[0]*100,5),'%='
              ,round(temp_lyft / init_df_lyft_shape[0]*100,5),'%='
              ,round(temp_yellow / init_df_yellow_shape[0]*100,5),'%' )

#記錄結束時間
end_time = time.time()
#計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

\=uber=lyft=yellow
data= 10 = 4 = 5882
占比= 1e-05 %= 1e-05 %= 0.03017 %

程式執行花費的時間： 16.2 秒
done時間: 2023-09-13 10:10:54.607693


In [15]:
#移除 (df_TLC['TripTime'] <= 0)
df_TLC = df_TLC.drop(df_TLC[df_TLC['TripTime'] <= 0].index)
#check-必須為0
print('TripTime<=0剩餘筆數:',(df_TLC['TripTime'] <= 0).sum())
print("\ndone時間:", datetime.datetime.now())

TripTime<=0剩餘筆數: 0

done時間: 2023-09-13 10:11:51.904357


### 移除TripMiles <= 0

In [16]:
#記錄開始時間
start_time = time.time()

#印出資訊
temp_uber = ((df_TLC['Name'] == 'uber') & (df_TLC['TripMiles'] <= 0)).sum()
temp_lyft = ((df_TLC['Name'] == 'lyft') & (df_TLC['TripMiles'] <= 0)).sum()
temp_yellow = ((df_TLC['Name'] == 'yellow') & (df_TLC['TripMiles'] <= 0)).sum()

print('\=uber=lyft=yellow')
print('data=',temp_uber,'=',temp_lyft,'=',temp_yellow)
print('占比=',round(temp_uber / init_df_uber_shape[0]*100,5),'%='
              ,round(temp_lyft / init_df_lyft_shape[0]*100,5),'%='
              ,round(temp_yellow / init_df_yellow_shape[0]*100,5),'%' )


#記錄結束時間
end_time = time.time()
#計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")

del temp_uber,temp_lyft,temp_yellow
print("done時間:",datetime.datetime.now())

\=uber=lyft=yellow
data= 16281 = 3860 = 243029
占比= 0.0195 %= 0.01217 %= 1.24674 %

程式執行花費的時間： 16.18 秒
done時間: 2023-09-13 10:12:08.149151


In [17]:
#移除 (df_TLC['TripMiles'] <= 0)
df_TLC = df_TLC.drop(df_TLC[df_TLC['TripMiles'] <= 0].index)
#check-必須為0
print('TripMiles<=0剩餘筆數:',(df_TLC['TripMiles'] <= 0).sum())
print("\ndone時間:", datetime.datetime.now())

TripMiles<=0剩餘筆數: 0

done時間: 2023-09-13 10:13:03.648514


### 新增節日相關資訊

In [18]:
# from holidays.calendars import gregorian
class AllHolidays(US):
    def _populate(self, year):
        super()._populate(year)
        self.pop_named("Lincoln's Birthday")
        self.pop_named("Susan B. Anthony Day")
        
        self._add_holiday("Valentine's Day", 2, 14)
        self._add_holiday("St.Patrick's Day", 3, 17)
        self._add_holiday("Halloween", 10, 31)
        
#        self._add_holiday("Thanksgiving weekend", 
#                          self._get_nth_weekday_of_month(month=11, n=4, weekday=4))
#        self._add_holiday("Thanksgiving weekend", 
#                          self._get_nth_weekday_of_month(month=11, n=4, weekday=5))
#        self._add_holiday("Thanksgiving weekend", 
#                          self._get_nth_weekday_of_month(month=11, n=4, weekday=6))
        
        self._add_holiday("Christmas Eve", 12, 24)
        self._add_holiday("Christmas season", 12, 26)
        self._add_holiday("Christmas season", 12, 27)
        self._add_holiday("Christmas season", 12, 28)
        self._add_holiday("Christmas season", 12, 29)
        self._add_holiday("Christmas season", 12, 30)
        
        self._add_holiday("New Year's Eve", 12, 31)

# 實例化AllHolidays
ny_holidays_all = AllHolidays(subdiv='NY', years=[2019, 2020, 2021, 2022, 2023])

# 新增2019~2023年的復活節、復活節星期一
ny_holidays_all[datetime.date(2019, 4, 21)] = "Easter"
ny_holidays_all[datetime.date(2019, 4, 22)] = "Easter Monday"
ny_holidays_all[datetime.date(2020, 4, 12)] = "Easter"
ny_holidays_all[datetime.date(2020, 4, 13)] = "Easter Monday"
ny_holidays_all[datetime.date(2021, 4, 4)] = "Easter"
ny_holidays_all[datetime.date(2021, 4, 5)] = "Easter Monday"
ny_holidays_all[datetime.date(2022, 4, 17)] = "Easter"
ny_holidays_all[datetime.date(2022, 4, 18)] = "Easter Monday"
ny_holidays_all[datetime.date(2023, 4, 9)] = "Easter"
ny_holidays_all[datetime.date(2023, 4, 10)] = "Easter Monday"

# 新增2019~2023感恩節
ny_holidays_all[datetime.date(2019,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 29)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 30)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 26)] =  "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 23)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 25)] = "Thanksgiving weekend"

# 查看
for i in sorted(ny_holidays_all.items()):
    print(i)

del i

(datetime.date(2019, 1, 1), "New Year's Day")
(datetime.date(2019, 1, 21), 'Martin Luther King Jr. Day')
(datetime.date(2019, 2, 14), "Valentine's Day")
(datetime.date(2019, 2, 18), "Washington's Birthday")
(datetime.date(2019, 3, 17), "St.Patrick's Day")
(datetime.date(2019, 4, 21), 'Easter')
(datetime.date(2019, 4, 22), 'Easter Monday')
(datetime.date(2019, 5, 27), 'Memorial Day')
(datetime.date(2019, 7, 4), 'Independence Day')
(datetime.date(2019, 9, 2), 'Labor Day')
(datetime.date(2019, 10, 14), 'Columbus Day')
(datetime.date(2019, 10, 31), 'Halloween')
(datetime.date(2019, 11, 5), 'Election Day')
(datetime.date(2019, 11, 11), 'Veterans Day')
(datetime.date(2019, 11, 28), 'Thanksgiving; Thanksgiving weekend')
(datetime.date(2019, 11, 29), 'Thanksgiving weekend')
(datetime.date(2019, 11, 30), 'Thanksgiving weekend')
(datetime.date(2019, 12, 24), 'Christmas Eve')
(datetime.date(2019, 12, 25), 'Christmas Day')
(datetime.date(2019, 12, 26), 'Christmas season')
(datetime.date(2019, 12, 

In [19]:
# 增加所有節日標籤
# all_data為Uber, Lyft, Yellow合併後的DataFrame，再請自行修改名稱
df_TLC['is_holiday'] = np.where(df_TLC['BoardingTime'].dt.date.isin(ny_holidays_all), True, False)
print(df_TLC['is_holiday'].value_counts())

print('-------------')
print('Name=True=False')
print('uber=',((df_TLC['Name'] == 'uber') & (df_TLC['is_holiday'] == True)).sum(),'=',((df_TLC['Name'] == 'uber') & (df_TLC['is_holiday'] == False)).sum())
print('lyft=',((df_TLC['Name'] == 'lyft') & (df_TLC['is_holiday'] == True)).sum(),'=',((df_TLC['Name'] == 'lyft') & (df_TLC['is_holiday'] == False)).sum())
print('yellow=',((df_TLC['Name'] == 'yellow') & (df_TLC['is_holiday'] == True)).sum(),'=',((df_TLC['Name'] == 'yellow') & (df_TLC['is_holiday'] == False)).sum())
print('Total=',(df_TLC['is_holiday'] == True).sum(),'=',(df_TLC['is_holiday'] == False).sum())

print("\ndone時間:", datetime.datetime.now())



is_holiday
False    127821619
True       6382532
Name: count, dtype: int64
-------------
Name=True=False
uber= 3993025 = 79489704
lyft= 1550064 = 30150880
yellow= 839443 = 18181035
Total= 6382532 = 127821619

done時間: 2023-09-13 10:14:06.146366


### 新增星期特徵
    #星期幾 (用上車時間 BoardingTime)
    #(一,二,三,四,五,六,日) = (0,1,2,3,4,5,6)

In [20]:
#記錄開始時間
start_time = time.time()

df_TLC['weekday'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.weekday
df_TLC['date'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.date
df_TLC['year'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.year
df_TLC['month'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.month
df_TLC['day'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.day
df_TLC['hour'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.hour

#記錄結束時間
end_time = time.time()
#計算程式執行花費的時間
execution_time = end_time - start_time
print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

程式執行花費的時間： 59.5 秒
done時間: 2023-09-13 10:15:05.661528


### 時間序列
    'Name','date','year','month','day','hour','weekday','PULocationID','DOLocationID',
    'TripMiles','TripTime','shared_match_flag'

In [21]:
#記錄開始時間
start_time = time.time()

dfTime_TLC = df_TLC.groupby(['Name','year','month','day',
                     'hour','weekday','PULocationID','is_holiday'
                  ]).size().reset_index(name="次數")

#記錄結束時間
end_time = time.time()
#計算程式執行花費的時間
execution_time = end_time - start_time
print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

程式執行花費的時間： 32.73 秒
done時間: 2023-09-13 10:15:38.422852


In [22]:
print('row=column')
print(dfTime_TLC.shape[0],'=',dfTime_TLC.shape[1])
dfTime_TLC

row=column
2541195 = 9


,Name,year,month,day,hour,weekday,PULocationID,is_holiday,次數
0,lyft,2023,1,1,0,6,2,True,1
1,lyft,2023,1,1,0,6,3,True,12
2,lyft,2023,1,1,0,6,4,True,49
3,lyft,2023,1,1,0,6,5,True,5
4,lyft,2023,1,1,0,6,6,True,3
...,...,...,...,...,...,...,...,...,...
2541190,yellow,2023,6,30,23,4,255,False,7
2541191,yellow,2023,6,30,23,4,256,False,7
2541192,yellow,2023,6,30,23,4,261,False,19
2541193,yellow,2023,6,30,23,4,262,False,18


In [23]:
print('row=column')
print(df_TLC.shape[0],'=',df_TLC.shape[1])
df_TLC

row=column
134204151 = 15


,Name,BoardingTime,DisembarkingTime,PULocationID,DOLocationID,TripMiles,TripTime,shared_match_flag,is_holiday,weekday,date,year,month,day,hour
0,uber,2023-01-01 00:19:38,2023-01-01 00:48:07,48,68,0.94,1709.0,N,True,6,2023-01-01,2023,1,1,0
1,uber,2023-01-01 00:58:39,2023-01-01 01:33:08,246,163,2.78,2069.0,N,True,6,2023-01-01,2023,1,1,0
2,uber,2023-01-01 00:20:27,2023-01-01 00:37:54,9,129,8.81,1047.0,N,True,6,2023-01-01,2023,1,1,0
3,uber,2023-01-01 00:41:05,2023-01-01 00:48:16,129,129,0.67,431.0,N,True,6,2023-01-01,2023,1,1,0
4,uber,2023-01-01 00:52:47,2023-01-01 01:04:51,129,92,4.38,724.0,N,True,6,2023-01-01,2023,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134705876,yellow,2023-06-30 23:30:21,2023-07-01 00:00:46,42,148,9.20,1825.0,N,False,4,2023-06-30,2023,6,30,23
134705877,yellow,2023-06-30 23:34:22,2023-07-01 00:32:59,132,74,20.20,3517.0,N,False,4,2023-06-30,2023,6,30,23
134705878,yellow,2023-06-30 23:45:00,2023-07-01 00:10:00,256,140,6.16,1500.0,N,False,4,2023-06-30,2023,6,30,23
134705879,yellow,2023-06-30 23:13:38,2023-06-30 23:51:19,91,246,11.65,2261.0,N,False,4,2023-06-30,2023,6,30,23


### 輸出為一份新檔案

In [24]:
custom_output_path = custom_output_path_dfTime

dfTime_TLC.to_parquet(f'{custom_output_path}/dfTime_TLC-{set_data_year}.parquet')
print("done時間:", datetime.datetime.now())

done時間: 2023-08-16 15:55:26.216569


In [25]:
custom_output_path = custom_output_path_dfTLC

df_TLC.to_parquet(f'{custom_output_path}/TLC-{set_data_year}.parquet')
print("done時間:", datetime.datetime.now())

done時間: 2023-08-16 15:56:23.781969
